In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [2]:
cust = pd.read_csv('data/Customers.csv')
trans = pd.read_csv('data/Transactions.csv')
prod = pd.read_csv('data/Products.csv')

In [4]:
cust.sample(5)

,CustomerID,CustomerName,Region,SignupDate
146,C0147,Hunter Fuller,South America,2022-05-16
95,C0096,Benjamin Mcclure,South America,2022-11-18
3,C0004,Kathleen Rodriguez,South America,2022-10-09
115,C0116,James Martinez,North America,2024-09-11
123,C0124,Lindsay Perez,Europe,2024-08-26


In [5]:
prod.sample(5)

,ProductID,ProductName,Category,Price
61,P062,HomeSense Novel,Books,374.16
3,P004,BookWorld Rug,Home Decor,95.69
17,P018,ComfortLiving Mystery Book,Books,436.89
53,P054,SoundWave Cookbook,Books,57.30
2,P003,ComfortLiving Biography,Books,44.12


In [6]:
trans.sample(5)

,TransactionID,CustomerID,ProductID,TransactionDate,Quantity,TotalValue,Price
357,T00979,C0112,P050,2024-05-08 03:42:39,3,1312.95,437.65
132,T00452,C0009,P060,2024-03-16 17:26:03,1,155.65,155.65
131,T00405,C0093,P060,2024-08-02 21:41:32,2,311.30,155.65
815,T00339,C0117,P019,2024-05-03 03:39:17,4,1541.48,385.37
934,T00611,C0084,P008,2024-12-16 14:32:21,2,293.70,146.85


### `Merging All the Datasets`

In [7]:
merged = trans.merge(cust, on='CustomerID').merge(prod, on='ProductID')

In [8]:
merged.sample(10)

,TransactionID,CustomerID,ProductID,TransactionDate,Quantity,TotalValue,Price_x,CustomerName,Region,SignupDate,ProductName,Category,Price_y
826,T00627,C0013,P020,2024-05-06 23:15:01,4,1585.36,396.34,Lauren Buchanan,South America,2024-05-19,ActiveWear Jacket,Clothing,396.34
210,T00023,C0012,P061,2024-06-18 10:55:39,4,627.84,156.96,Kevin May,South America,2024-08-07,HomeSense Desk Lamp,Home Decor,156.96
218,T00655,C0182,P061,2024-04-28 02:14:05,4,627.84,156.96,Joshua Preston,Europe,2024-12-28,HomeSense Desk Lamp,Home Decor,156.96
276,T00524,C0014,P047,2024-01-17 18:31:55,2,318.66,159.33,Deborah Wilcox,Europe,2024-06-22,ComfortLiving Cookware Set,Home Decor,159.33
109,T00533,C0191,P062,2024-07-12 10:33:55,4,1496.64,374.16,Samantha Gibson DVM,South America,2024-04-07,HomeSense Novel,Books,374.16
801,T00473,C0066,P080,2024-03-13 04:00:25,2,607.00,303.50,Catherine White,Europe,2024-11-15,ActiveWear Jacket,Clothing,303.50
121,T00231,C0069,P041,2024-08-27 19:54:49,2,912.56,456.28,Stacy Foster,Europe,2023-12-03,ComfortLiving Smartphone,Electronics,456.28
705,T00234,C0176,P090,2024-07-07 23:53:12,4,661.20,165.30,Nicole Booth,North America,2022-03-23,ComfortLiving Biography,Books,165.30
176,T00910,C0071,P040,2024-05-22 03:47:06,1,153.19,153.19,Taylor Murphy,South America,2022-07-01,SoundWave Mystery Book,Books,153.19
730,T00887,C0066,P083,2024-08-28 20:52:33,3,1367.16,455.72,Catherine White,Europe,2024-11-15,ActiveWear Smartwatch,Electronics,455.72


### `Total Spend and Purchase info per Customer`

In [9]:
customer_features = merged.groupby('CustomerID').agg({
    'TotalValue': ['sum', 'mean'],
    'ProductID': 'count',
}).reset_index()
customer_features.columns = ['CustomerID', 'TotalSpend', 'AverageSpend', 'TransactionCount']

In [10]:
customer_features.sample(5)

,CustomerID,TotalSpend,AverageSpend,TransactionCount
35,C0036,1877.96,625.986667,3
174,C0175,6210.53,621.053000,10
118,C0119,1191.61,238.322000,5
166,C0167,2300.42,766.806667,3
52,C0053,5316.46,886.076667,6


In [11]:
encoder = OneHotEncoder()
region_encoded = encoder.fit_transform(cust[['Region']]).toarray()
region_df = pd.DataFrame(region_encoded, columns=encoder.get_feature_names_out(['Region']))
customers = pd.concat([cust, region_df], axis=1)

In [12]:
region_df.head(10)

,Region_Asia,Region_Europe,Region_North America,Region_South America
0,0.0,0.0,0.0,1.0
1,1.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,1.0
4,1.0,0.0,0.0,0.0
5,0.0,0.0,0.0,1.0
6,1.0,0.0,0.0,0.0
7,0.0,0.0,1.0,0.0
8,0.0,1.0,0.0,0.0
9,0.0,1.0,0.0,0.0


### `Final Calcultaion Matrix`

In [13]:

customer_features = customer_features.merge(customers, on='CustomerID')

In [14]:

product_matrix = pd.pivot_table(
    merged, index='CustomerID', columns='ProductID', values='Quantity', aggfunc='sum', fill_value=0
)

In [15]:
final_features = customer_features.set_index('CustomerID').drop(columns=['Region']).join(product_matrix)

In [16]:

final_features.head()

,TotalSpend,AverageSpend,TransactionCount,CustomerName,SignupDate,Region_Asia,Region_Europe,Region_North America,Region_South America,P001,...,P091,P092,P093,P094,P095,P096,P097,P098,P099,P100
CustomerID,,,,,,,,,,,,,,,,,,,,,
C0001,3354.52,670.904,5,Lawrence Carroll,2022-07-10,0.0,0.0,0.0,1.0,0,...,0,0,0,0,0,2,0,0,0,0
C0002,1862.74,465.685,4,Elizabeth Lutz,2022-02-13,1.0,0.0,0.0,0.0,0,...,0,0,0,0,2,0,0,0,0,0
C0003,2725.38,681.345,4,Michael Rivera,2024-03-07,0.0,0.0,0.0,1.0,0,...,0,0,0,0,0,0,0,0,0,0
C0004,5354.88,669.360,8,Kathleen Rodriguez,2022-10-09,0.0,0.0,0.0,1.0,0,...,0,0,0,0,0,0,3,0,0,0
C0005,2034.24,678.080,3,Laura Weber,2022-08-15,1.0,0.0,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:

final_features_cleaned = final_features.select_dtypes(include=[np.number])

`StandardScaling`

In [18]:
scaler = StandardScaler()
final_features_scaled = scaler.fit_transform(final_features_cleaned)

`Similarity Score Calculation`

In [21]:
similarity_matrix = cosine_similarity(final_features_scaled)

In [22]:
similarity_df = pd.DataFrame(similarity_matrix, index=final_features.index, columns=final_features.index)

In [23]:

similarity_df.head()

CustomerID,C0001,C0002,C0003,C0004,C0005,C0006,C0007,C0008,C0009,C0010,...,C0191,C0192,C0193,C0194,C0195,C0196,C0197,C0198,C0199,C0200
CustomerID,,,,,,,,,,,,,,,,,,,,,
C0001,1.000000,-0.062650,-0.017169,-0.045244,-0.070553,-0.007035,0.127006,-0.080208,-0.068267,-0.064164,...,0.072788,0.010273,-0.073803,0.361986,-0.035882,-0.071698,-0.071303,-0.063227,0.301482,-0.081469
C0002,-0.062650,1.000000,-0.041138,-0.069248,0.055868,-0.067886,0.027814,0.218993,0.041706,-0.019137,...,-0.045219,-0.021767,0.019434,-0.059284,-0.065237,-0.067662,-0.022716,0.027555,-0.026624,-0.011284
C0003,-0.017169,-0.041138,1.000000,0.053504,0.219818,-0.006169,-0.038919,0.207102,-0.035381,-0.042219,...,-0.009913,0.009514,-0.050354,-0.072361,-0.029737,-0.051637,-0.044086,-0.028917,-0.054648,-0.059878
C0004,-0.045244,-0.069248,0.053504,1.000000,0.042745,-0.036028,-0.069941,-0.000824,-0.132555,0.020872,...,-0.038231,-0.043371,-0.070582,0.011362,-0.036126,-0.061909,-0.086356,-0.128107,0.099649,-0.060114
C0005,-0.070553,0.055868,0.219818,0.042745,1.000000,-0.050691,0.059169,-0.083768,0.040299,-0.026893,...,-0.053635,0.204579,0.040547,0.071039,-0.079281,-0.045016,-0.015665,0.044992,-0.033076,0.006145


`Getting top 3 similar customers`

In [24]:

def get_top_lookalikes(customer_id, sim_df, top_n=3):
    similar_customers = sim_df.loc[customer_id].sort_values(ascending=False)[1:top_n+1]
    return [(index, score) for index, score in similar_customers.items()]

In [25]:

lookalike_results = {}
for customer_id in final_features.index[:20]:
    lookalike_results[customer_id] = get_top_lookalikes(customer_id, similarity_df)

In [26]:

lookalike_list = []
for cust_id, lookalikes in lookalike_results.items():
    lookalike_list.append({
        'cust_id': cust_id,
        'lookalikes': lookalikes
    })

In [27]:

lookalike_df = pd.DataFrame(lookalike_list)

In [29]:
lookalike_df.head()

,cust_id,lookalikes
0,C0001,"[(C0104, 0.3883265901729216), (C0194, 0.361986..."
1,C0002,"[(C0030, 0.38121945722789435), (C0091, 0.36211..."
2,C0003,"[(C0181, 0.4906014700685738), (C0134, 0.444807..."
3,C0004,"[(C0070, 0.32404774520643487), (C0175, 0.31814..."
4,C0005,"[(C0096, 0.4175298687419521), (C0023, 0.410275..."


In [30]:

# Calculate Lifetime Value (LTV)
customer_ltv = trans.groupby('CustomerID')['TotalValue'].sum().reset_index()
customer_ltv.columns = ['CustomerID', 'LTV']

In [31]:
# Calculate Recency (Days since last purchase)
trans['TransactionDate'] = pd.to_datetime(trans['TransactionDate'])
last_purchase = trans.groupby('CustomerID')['TransactionDate'].max().reset_index()
last_purchase['Recency'] = (pd.to_datetime('today') - last_purchase['TransactionDate']).dt.days
last_purchase = last_purchase[['CustomerID', 'Recency']]

In [32]:
# Merge LTV and Recency into the customers DataFrame
cust = pd.merge(cust, customer_ltv, on='CustomerID', how='left')
cust = pd.merge(cust, last_purchase, on='CustomerID', how='left')

In [34]:
final_features = cust[['LTV', 'Recency']]

In [35]:

scaler = StandardScaler()
final_features_scaled = scaler.fit_transform(final_features)

In [36]:

# Check for missing values in the final features
final_features.isnull().sum()

LTV        1
Recency    1
dtype: int64

`Filling missing values`

In [37]:

# Fill missing values with the median of each column
final_features_filled = final_features.fillna(final_features.median())

# Check if any missing values remain
final_features_filled.isnull().sum()

LTV        0
Recency    0
dtype: int64

In [38]:

scaler = StandardScaler()
final_features_scaled = scaler.fit_transform(final_features_filled)

In [39]:

final_features_scaled[:5]

array([[-0.0609474 , -0.26877595],
       [-0.87897079, -0.69371815],
       [-0.40593878,  0.72275587],
       [ 1.03595782, -0.97701296],
       [-0.78492809, -0.28294069]])

In [40]:
similarity_matrix = cosine_similarity(final_features_scaled)

In [42]:
similarity_df = pd.DataFrame(similarity_matrix, index=customers['CustomerID'], columns=customers['CustomerID'])
similarity_df.head()

CustomerID,C0001,C0002,C0003,C0004,C0005,C0006,C0007,C0008,C0009,C0010,...,C0191,C0192,C0193,C0194,C0195,C0196,C0197,C0198,C0199,C0200
CustomerID,,,,,,,,,,,,,,,,,,,,,
C0001,1.000000,0.777784,-0.742009,0.508236,0.538754,-0.469421,-0.686060,0.775696,0.191395,0.625984,...,0.996824,-0.672085,-0.940187,0.195700,0.746631,0.548958,0.898524,0.120845,0.399944,0.586715
C0002,0.777784,1.000000,-0.155761,-0.146004,0.948549,-0.920085,-0.076323,0.206654,0.765775,0.977032,...,0.825366,-0.057326,-0.517148,-0.464166,0.162595,-0.098387,0.974735,0.717916,0.887144,-0.052643
C0003,-0.742009,-0.155761,1.000000,-0.954468,0.165019,-0.243623,0.996802,-0.998661,0.515980,0.058308,...,-0.686267,0.995099,0.926001,-0.802638,-0.999976,-0.967677,-0.372463,0.575809,0.317678,-0.978225
C0004,0.508236,-0.146004,-0.954468,1.000000,-0.451730,0.521856,-0.975255,0.937757,-0.748023,-0.353460,...,0.438042,-0.979287,-0.771219,0.944027,0.952380,0.998849,0.078654,-0.793488,-0.586074,0.995598
C0005,0.538754,0.948549,0.165019,-0.451730,1.000000,-0.996776,0.243312,-0.113775,0.930003,0.994234,...,0.604130,0.261734,-0.219536,-0.720739,-0.158189,-0.408420,0.853860,0.901395,0.987622,-0.366126


In [44]:

from scipy.stats import zscore

In [45]:

trans['TotalValue_zscore'] = zscore(trans['TotalValue'])
trans['Quantity_zscore'] = zscore(trans['Quantity'])

`Removing Outliers`

In [46]:
transactions_cleaned = trans[(trans['TotalValue_zscore'].abs() <= 3) &
                                    (trans['Quantity_zscore'].abs() <= 3)]

In [54]:
cust.head()

,CustomerID,CustomerName,Region,SignupDate,LTV,Recency
0,C0001,Lawrence Carroll,South America,2022-07-10,3354.52,86.0
1,C0002,Elizabeth Lutz,Asia,2022-02-13,1862.74,56.0
2,C0003,Michael Rivera,South America,2024-03-07,2725.38,156.0
3,C0004,Kathleen Rodriguez,South America,2022-10-09,5354.88,36.0
4,C0005,Laura Weber,Asia,2022-08-15,2034.24,85.0


In [47]:

transactions_cleaned.head()

,TransactionID,CustomerID,ProductID,TransactionDate,Quantity,TotalValue,Price,TotalValue_zscore,Quantity_zscore
0,T00001,C0199,P067,2024-08-25 12:38:23,1,300.68,300.68,-0.789850,-1.375488
1,T00112,C0146,P067,2024-05-27 22:23:54,1,300.68,300.68,-0.789850,-1.375488
2,T00166,C0127,P067,2024-04-25 07:38:55,1,300.68,300.68,-0.789850,-1.375488
3,T00272,C0087,P067,2024-03-26 22:55:37,2,601.36,300.68,-0.179825,-0.480571
4,T00363,C0070,P067,2024-03-21 15:10:10,3,902.04,300.68,0.430200,0.414347


In [49]:

customers_cleaned = cust[(cust['LTV'] > cust['LTV'].quantile(0.01)) &
                            (cust['Recency'] < cust['Recency'].quantile(0.99))]

customers_cleaned.head()

,CustomerID,CustomerName,Region,SignupDate,LTV,Recency
0,C0001,Lawrence Carroll,South America,2022-07-10,3354.52,86.0
1,C0002,Elizabeth Lutz,Asia,2022-02-13,1862.74,56.0
2,C0003,Michael Rivera,South America,2024-03-07,2725.38,156.0
3,C0004,Kathleen Rodriguez,South America,2022-10-09,5354.88,36.0
4,C0005,Laura Weber,Asia,2022-08-15,2034.24,85.0


In [50]:

from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
    

In [51]:
final_features_sparse = csr_matrix(final_features_scaled)
similarity_matrix_sparse = cosine_similarity(final_features_sparse)
similarity_df_sparse = pd.DataFrame(similarity_matrix_sparse,
                                    index=customers['CustomerID'], columns=customers['CustomerID'])

similarity_df_sparse.head()

CustomerID,C0001,C0002,C0003,C0004,C0005,C0006,C0007,C0008,C0009,C0010,...,C0191,C0192,C0193,C0194,C0195,C0196,C0197,C0198,C0199,C0200
CustomerID,,,,,,,,,,,,,,,,,,,,,
C0001,1.000000,0.777784,-0.742009,0.508236,0.538754,-0.469421,-0.686060,0.775696,0.191395,0.625984,...,0.996824,-0.672085,-0.940187,0.195700,0.746631,0.548958,0.898524,0.120845,0.399944,0.586715
C0002,0.777784,1.000000,-0.155761,-0.146004,0.948549,-0.920085,-0.076323,0.206654,0.765775,0.977032,...,0.825366,-0.057326,-0.517148,-0.464166,0.162595,-0.098387,0.974735,0.717916,0.887144,-0.052643
C0003,-0.742009,-0.155761,1.000000,-0.954468,0.165019,-0.243623,0.996802,-0.998661,0.515980,0.058308,...,-0.686267,0.995099,0.926001,-0.802638,-0.999976,-0.967677,-0.372463,0.575809,0.317678,-0.978225
C0004,0.508236,-0.146004,-0.954468,1.000000,-0.451730,0.521856,-0.975255,0.937757,-0.748023,-0.353460,...,0.438042,-0.979287,-0.771219,0.944027,0.952380,0.998849,0.078654,-0.793488,-0.586074,0.995598
C0005,0.538754,0.948549,0.165019,-0.451730,1.000000,-0.996776,0.243312,-0.113775,0.930003,0.994234,...,0.604130,0.261734,-0.219536,-0.720739,-0.158189,-0.408420,0.853860,0.901395,0.987622,-0.366126


### `Final Similarity calculation`

In [55]:

recommendations = {}

for cust_id in customers['CustomerID']:
    similar_customers = similarity_df[cust_id].sort_values(ascending=False).iloc[1:4]
    recommendations[cust_id] = list(zip(similar_customers.index, similar_customers.values))

lookalike_recommendations = pd.DataFrame(list(recommendations.items()), columns=['cust_id', 'lookalikes'])
lookalike_recommendations = lookalike_recommendations[0:20]

In [56]:
lookalike_recommendations

,cust_id,lookalikes
0,C0001,"[(C0115, 0.9991730564412933), (C0092, 0.998706..."
1,C0002,"[(C0044, 0.9998847293637366), (C0172, 0.999778..."
2,C0003,"[(C0120, 0.9999929162555088), (C0112, 0.999568..."
3,C0004,"[(C0017, 0.9999506840286218), (C0101, 0.999897..."
4,C0005,"[(C0123, 0.9999197163676554), (C0032, 0.999186..."
5,C0006,"[(C0124, 0.9999315988299557), (C0087, 0.999877..."
6,C0007,"[(C0192, 0.9998187539325601), (C0097, 0.999459..."
7,C0008,"[(C0183, 0.9998930120733044), (C0195, 0.998995..."
8,C0009,"[(C0140, 0.9989251489403402), (C0027, 0.998435..."
9,C0010,"[(C0019, 0.9999962827851769), (C0177, 0.999135..."


In [57]:

lookalike_recommendations.to_csv('Refined_Lookalike_Recommendations.csv', index=False)
print("Refined_Lookalike_Recommendations.csv has been saved!")

Refined_Lookalike_Recommendations.csv has been saved!
